In [1]:
import scrapy
import logging
import pandas as pd
from datetime import date
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [2]:
class PatientSpider(scrapy.Spider):
    name = "patient_check"
    start_urls = ['https://ncov.moh.gov.vn/web/guest/trang-chu']
    rows = []
    out = 'MOH_PatientTable_' + date.today().strftime("%d_%m") + '.csv'

    def __init__(self):
        logging.getLogger('scrapy').propagate = False

    def close(self, reason):
        df_out = pd.DataFrame(self.rows)[::-1]
        df_out['Patient'] = df_out['Patient'].str[2:]
        df_out.to_csv(self.out,index=False)
        print("Hoàn tất. Dữ liệu đã được xuất ra file " + self.out)

    def parse(self, response):
        for patient in response.xpath('//*[@class="table table-striped table-covid19"]//tbody/tr'):
            p_dict={'Patient': patient.xpath('td[1]//text()').extract_first(),
                    'Age': patient.xpath('td[2]//text()').extract_first(),
                    'Location': patient.xpath('td[3]//text()').extract_first(),
                    'Status': patient.xpath('td[4]//text()').extract_first(),
                    'Nationality': patient.xpath('td[5]//text()').extract_first()}
            self.rows += [p_dict]


In [3]:
process = CrawlerProcess(get_project_settings())
process.crawl(PatientSpider)
process.start()

2021-06-14 16:19:18 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-06-14 16:19:18 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.3 (default, Mar 27 2019, 16:54:48) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.6.1, Platform Darwin-18.7.0-x86_64-i386-64bit
2021-06-14 16:19:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-06-14 16:19:18 [scrapy.crawler] INFO: Overridden settings:
{}
2021-06-14 16:19:18 [scrapy.extensions.telnet] INFO: Telnet Password: 51f47b49d7f38e4e
2021-06-14 16:19:18 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']


Hoàn tất. Dữ liệu đã được xuất ra file MOH_PatientTable_14_06.csv
